# Mission to Mars

In [24]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
from pprint import pprint

# Step 1 - Scraping

## NASA Mars News
- setup splinter and open the browser
- get the first title and paragraph

In [29]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

#url of the page to be scraped
url = 'https://mars.nasa.gov/news/'

#get splinter to visit the url
browser.visit(url)

#create a soup object and parse it
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/johnnyshuford/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [7]:
#get the first title
titles = soup.find_all('div', class_='content_title', limit = 2)
news_title = titles[1].text

In [8]:
#get the first paragraph
pargs = soup.find_all('div', class_='article_teaser_body', limit = 1)
news_p = pargs[0].text

In [9]:
#quit the running browser
browser.quit()

## JPL Mars Space Images - Featured Image
- opening the browser
- getting the featured image

In [10]:
#opening the browser
browser = Browser('chrome', **executable_path, headless=False)

#url of the page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

#get splinter to visit the url
browser.visit(url)

#create a soup object and parse it
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [11]:
#navigate to the right page using splinter
browser.links.find_by_partial_text('FULL IMAGE').click()
browser.links.find_by_partial_text('more info').click()

In [12]:
#get the image url
img_url = soup.find('article', class_='carousel_item')['style'].replace('background-image: url(','').replace(');','')[1:-1]

In [13]:
print(img_url)

/spaceimages/images/wallpaper/PIA16883-1920x1200.jpg


In [14]:
#quit the running browser
browser.quit()

## Mars Facts
- scrape using read_html
- create a data frame
- convert to html table string

In [15]:
#url of the page to be scraped
url = 'https://space-facts.com/mars/'

In [16]:
#perform the scrape
table = pd.read_html(url)

In [17]:
#creat a dataframe from the scrape
df = table[0]

In [18]:
#get the HTML table string
html_table = df.to_html()

## Mars Hemispheres
- opening the browser
- looping through the websitet to grab the title and URL

In [26]:
#opening the browser
browser = Browser('chrome', **executable_path, headless=False)

#url of the page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

#get splinter to visit the url
browser.visit(url)

In [27]:
links = browser.find_by_css('a.product-item h3')

hemisphere_image_urls = []

main_url = 'https://astrogeology.usgs.gov'
for x in range(len(links)):
    mars = {}
    
    browser.find_by_css('a.product-item h3')[x].click()
    
    soup = BeautifulSoup(browser.html, 'html.parser')
    
    mars['title'] = soup.find('h2', class_ = 'title').text
    
    browser.links.find_by_partial_text('Open').click()
    
    mars['img_url'] = main_url + str(soup.find('img', class_='wide-image')['src'])
    
    hemisphere_image_urls.append(mars)
    
    browser.visit(url)
    
browser.quit()

In [28]:
mars_dict = {
        'news title': news_title,
        'news paragraph': news_p,
        'featured img': img_url,
        'table': html_table,
        'hemispheres' : hemisphere_image_urls
    }

pprint(mars_dict)

{'featured img': '/spaceimages/images/wallpaper/PIA16883-1920x1200.jpg',
 'hemispheres': [{'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
                  'title': 'Cerberus Hemisphere Enhanced'},
                 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
                  'title': 'Schiaparelli Hemisphere Enhanced'},
                 {'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
                  'title': 'Syrtis Major Hemisphere Enhanced'},
                 {'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
                  'title': 'Valles Marineris Hemisphere Enhanced'}],
 'news paragraph': "NASA's Perseverance rover carries a device to convert "
                   'Ma